# Fase 1 (Continuação): Codificação de Variáveis

Este notebook dá continuidade à fase de pré-processamento, focando na transformação de variáveis categóricas em representações numéricas para que possam ser utilizadas por algoritmos de Machine Learning.

As seguintes estratégias de codificação serão aplicadas:

1.  **Variável Alvo:** Transformação da coluna `Churn` para o formato `0/1`.
2.  **Variáveis Binárias:** Conversão de colunas com duas categorias (ex: 'Yes'/'No') para `0/1`.
3.  **Variáveis Ordinais:** Mapeamento da coluna `Contract` para valores numéricos que preservem sua ordem intrínseca.
4.  **Variáveis Nominais:** Aplicação do método One-Hot Encoding para as demais variáveis categóricas sem ordem específica.
5.  **Salvamento:** O DataFrame final, totalmente pré-processado, será salvo em um novo arquivo.

In [7]:
import pandas as pd
import numpy as np

### Carregando os Dados Limpos

Iniciamos carregando o dataset que foi limpo na etapa anterior.

In [8]:
# Caminho do arquivo limpo gerado pelo notebook anterior
caminho_arquivo_limpo = r'E:\Documentos\Cursos\Alura\ONE\Desafio-03\telecom_x_cleaned.csv'

# Carregar o dataset
df = pd.read_csv(caminho_arquivo_limpo)

# Exibir as primeiras linhas e os tipos de dados para verificar o estado atual
print("Dataset antes da codificação:")
display(df.head())
print("\nTipos de dados antes da codificação:")
df.info()

Dataset antes da codificação:


,Churn,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Charges.Monthly,Charges.Total
0,No,Female,0,Yes,Yes,9,Yes,No,DSL,No,Yes,No,Yes,Yes,No,One year,Yes,Mailed check,65.6,593.30
1,No,Male,0,No,No,9,Yes,Yes,DSL,No,No,No,No,No,Yes,Month-to-month,No,Mailed check,59.9,542.40
2,Yes,Male,0,No,No,4,Yes,No,Fiber optic,No,No,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.9,280.85
3,Yes,Male,1,Yes,No,13,Yes,No,Fiber optic,No,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,98.0,1237.85
4,Yes,Female,1,Yes,No,3,Yes,No,Fiber optic,No,No,No,Yes,Yes,No,Month-to-month,Yes,Mailed check,83.9,267.40



Tipos de dados antes da codificação:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Churn             7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  Pape

### 1. Codificação da Variável Alvo (`Churn`)

A variável alvo `Churn` precisa ser convertida para um formato numérico. Adotaremos a convenção `No = 0` e `Yes = 1`.

In [9]:
df['Churn'] = df['Churn'].map({'No': 0, 'Yes': 1})

print("Valores da coluna 'Churn' após codificação:")
print(df['Churn'].value_counts())

Valores da coluna 'Churn' após codificação:
Churn
0    5174
1    1869
Name: count, dtype: int64


### 2. Codificação de Variáveis Binárias

Colunas com respostas 'Yes'/'No' e a coluna `gender` serão convertidas para `0/1`.

In [10]:
# Codificação para 'gender' especificamente
df['gender'] = df['gender'].map({'Female': 0, 'Male': 1})

# Lista de outras colunas binárias com formato 'Yes'/'No'
colunas_binarias = ['Partner', 'Dependents', 'PhoneService', 'PaperlessBilling']
mapa_binario = {'No': 0, 'Yes': 1}

for col in colunas_binarias:
    df[col] = df[col].map(mapa_binario)

print("Prévia dos dados após codificação binária:")
display(df[['gender', 'Partner', 'Dependents', 'PhoneService', 'PaperlessBilling']].head())

Prévia dos dados após codificação binária:


,gender,Partner,Dependents,PhoneService,PaperlessBilling
0,0,1,1,1,1
1,1,0,0,1,0
2,1,0,0,1,1
3,1,1,0,1,1
4,0,1,0,1,1


### 3. Codificação de Variável Ordinal (`Contract`)

A coluna `Contract` tem uma ordem clara (`Month-to-month` < `One year` < `Two year`), que representa um nível de compromisso crescente do cliente. Usaremos o mapeamento manual para preservar essa relação.

In [11]:
mapa_contrato = {
    'Month-to-month': 0,
    'One year': 1,
    'Two year': 2
}

df['Contract'] = df['Contract'].map(mapa_contrato)

print("Valores da coluna 'Contract' após codificação ordinal:")
print(df['Contract'].value_counts())

Valores da coluna 'Contract' após codificação ordinal:
Contract
0    3875
2    1695
1    1473
Name: count, dtype: int64


### 4. Codificação de Variáveis Nominais (One-Hot Encoding)

As variáveis categóricas restantes não possuem uma ordem intrínseca. Para estas, aplicaremos o One-Hot Encoding, que cria uma nova coluna para cada categoria.

Usaremos `pd.get_dummies` com o parâmetro `drop_first=True` para evitar multicolinearidade, uma boa prática para modelos lineares.

In [12]:
# Identificar as colunas categóricas restantes (tipo 'object')
colunas_nominais = df.select_dtypes(include='object').columns.tolist()

print(f"Colunas a serem transformadas com One-Hot Encoding: {colunas_nominais}")

# Aplicar One-Hot Encoding
df = pd.get_dummies(df, columns=colunas_nominais, drop_first=True)

print("\nDimensões do dataset após One-Hot Encoding:", df.shape)
print("Prévia dos dados com as novas colunas:")
display(df.head())

Colunas a serem transformadas com One-Hot Encoding: ['MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'PaymentMethod']

Dimensões do dataset após One-Hot Encoding: (7043, 30)
Prévia dos dados com as novas colunas:


,Churn,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,Contract,PaperlessBilling,Charges.Monthly,...,DeviceProtection_Yes,TechSupport_No internet service,TechSupport_Yes,StreamingTV_No internet service,StreamingTV_Yes,StreamingMovies_No internet service,StreamingMovies_Yes,PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check
0,0,0,0,1,1,9,1,1,1,65.6,...,False,False,True,False,True,False,False,False,False,True
1,0,1,0,0,0,9,1,0,0,59.9,...,False,False,False,False,False,False,True,False,False,True
2,1,1,0,0,0,4,1,0,1,73.9,...,True,False,False,False,False,False,False,False,True,False
3,1,1,1,1,0,13,1,0,1,98.0,...,True,False,False,False,True,False,True,False,True,False
4,1,0,1,1,0,3,1,0,1,83.9,...,False,False,True,False,True,False,False,False,False,True


### 5. Variáveis Numéricas

As variáveis `tenure`, `Charges.Monthly` e `Charges.Total` já estão no formato numérico correto. A padronização (scaling) dessas variáveis será feita em uma etapa posterior, durante a fase de modelagem. 

**Motivo:** A padronização deve ser "aprendida" (fitted) apenas no conjunto de treino e depois aplicada ao conjunto de teste para evitar vazamento de dados (data leakage), garantindo que o modelo seja avaliado em dados verdadeiramente "novos".

### 6. Salvando o Dataset Final Pré-processado

O DataFrame agora está totalmente codificado e pronto para a modelagem. Vamos salvá-lo em um novo arquivo.

In [13]:
caminho_final = r'E:\Documentos\Cursos\Alura\ONE\Desafio-03\telecom_x_processed.csv'

df.to_csv(caminho_final, index=False)

print(f"Dataset final pré-processado e salvo com sucesso em: {caminho_final}")
print(f"Dimensões finais: {df.shape}")

Dataset final pré-processado e salvo com sucesso em: E:\Documentos\Cursos\Alura\ONE\Desafio-03\telecom_x_processed.csv
Dimensões finais: (7043, 30)
